In [24]:
import os
from time import sleep

import pandas as pd
import phoenix as px
from datasets import Dataset
from dotenv import load_dotenv
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from phoenix.session.evaluation import get_qa_with_reference
from phoenix.trace import SpanEvaluations, using_project
from phoenix.trace.langchain import LangChainInstrumentor
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness, answer_correctness,
)

In [25]:
# load .env file
load_dotenv()

# get azure credentials from .env file
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
temperature = os.getenv("TEMPERATURE")

In [26]:
def build_chain(embeddings_model, model):
    # load vectorstore
    vectorstore = Chroma(embedding_function=embeddings_model, persist_directory="./chroma_db")

    # initialize a retriever from the vectorstore
    retriever = vectorstore.as_retriever()

    # crate a system prompt that tells the LLM to answer questions based on the given context
    # and use a variable that represents the context
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
    )

    # create a prompt template with the system prompt
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    # create a helper chain that inserts the retrieved documents into the prompt
    question_answer_chain = create_stuff_documents_chain(model, prompt)

    # create the final RAG chai
    chain = create_retrieval_chain(retriever, question_answer_chain)

    return chain

In [27]:
def generate_ragas_dataset(chain, test_data_df):
    test_questions = test_data_df["question"].values

    # execute chain and store answers and retrieved context
    responses = [
        chain.invoke({"input": question})
        for question in test_questions
    ]

    contexts = []
    for response in responses:
        page_contents = [doc.page_content for doc in response["context"]]
        contexts.append(page_contents)

    test_data_df["answer"] = [response["answer"] for response in responses]
    test_data_df["contexts"] = contexts
    test_dataset = Dataset.from_pandas(test_data_df)

    return test_dataset

In [28]:
# initialize the Embedding Model
embeddings = AzureOpenAIEmbeddings(
    api_version=api_version,
    openai_api_type='azure',
    azure_endpoint=azure_endpoint,
    azure_deployment=embedding_deployment_name,
)

# initialize the Azure OpenAI Model
model = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    deployment_name=deployment_name,
    api_key=api_key,
    api_version=api_version,
    openai_api_type="azure",
    temperature=0.0,
    streaming=True,
)

In [29]:
# create a chain
chain = build_chain(embeddings, model)

In [30]:
# read in the testset as a dataframe
evaluation_data = pd.read_csv(
    "testdata.csv",
    usecols=["question", "ground_truth"],
)

In [31]:
# start phoenix session and client
session = px.launch_app(use_temp_dir=False)
client = px.Client()

# initialize Langchain auto-instrumentation
LangChainInstrumentor().instrument()

WARNI [phoenix.session.session] Existing running Phoenix instance detected! Shutting it down and starting a new instance...
WARNI [opentelemetry.instrumentation.instrumentor] Attempting to instrument while already instrumented


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
💽 Your data is being persisted to sqlite:///C:\Users\Johanna Heinz\.phoenix/phoenix.db
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [32]:
# create ragas testset in an extra project
with using_project("test"):
    ragas_eval_dataset = generate_ragas_dataset(chain, evaluation_data)

ragas_evals_df = pd.DataFrame(ragas_eval_dataset)

ragas_evals_df

,question,ground_truth,answer,contexts
0,Wie bringt die interne Datenakademie der Merck...,Die interne Datenakademie der Merck KGaA bring...,"Boris Adryan, der Akademische Direktor der int...",[Wieviel KI müssen wir unseren Kunden erklären...
1,Wie hat die Einführung der Transformer-Archite...,Nach der Einführung der Transformer-Architektu...,Die Einführung der Transformer-Architektur dur...,[Eine Einführung in Large Language Models\n\nC...
2,Wie können Data Scientists realistisches Erwar...,Data Scientists können realistisches Erwartung...,Data Scientists können realistisches Erwartung...,[KI Power-Play: Realistisches Erwartungsmanage...
3,Wie tragen Data-Governance und MLOps-Techniken...,Data-Governance und MLOps-Techniken tragen zur...,"Die Anwendung von etablierten Data-Governance,...",[Die Engineering-Perspektive auf den EU AI Act...
4,Wie wird die Sentimentanalyse mit LLMs im Work...,Die Sentimentanalyse wird im Workshop mit LLMs...,Im Workshop wird die Sentimentanalyse mit LLMs...,[Datenanalyse mit Machine Learning\n\nIn diese...
5,Welche 4 Risikogruppen für KI-Systeme erforder...,Die vier Risikogruppen für KI-Systeme sind ver...,Der EU AI Act kategorisiert KI-Systeme in vier...,[Die Engineering-Perspektive auf den EU AI Act...
6,Wie verbessert Christian Winkler die UX in NLP...,Christian Winkler konzentriert sich auf die Op...,Christian Winkler beschäftigt sich als Profess...,[Schluss mit Prototyp: GPTx und RAG in der Pra...
7,How do data availability and explainable AI im...,The context discusses the impact of data avail...,The increasing availability of data and data e...,[Datenanalysen im Fußball – ein Überblick\n\nS...
8,How has Kira Engelhardt's finance background a...,Kira Engelhardt's finance background and leade...,The provided context does not provide a direct...,[KI Power-Play: Realistisches Erwartungsmanage...
9,Welche technischen Anforderungen gibt es für T...,"Die technischen Anforderungen für Teilnehmer, ...","Teilnehmer sollten einen Laptop mitbringen, de...",[Datenanalyse mit Machine Learning\n\nIn diese...


In [33]:
# wait a few seconds in case data hasn't become fully available yet
sleep(5)

# collect information about rag spans
spans_dataframe = get_qa_with_reference(client, project_name="test")

spans_dataframe

,input,output,reference
context.span_id,,,
ad86cab1546cbb01,Wie bringt die interne Datenakademie der Merck...,"{""input"": ""Wie bringt die interne Datenakademi...",Wieviel KI müssen wir unseren Kunden erklären?...
f7708d5ab80afdbf,Wie hat die Einführung der Transformer-Archite...,"{""input"": ""Wie hat die Einführung der Transfor...",Eine Einführung in Large Language Models\n\nCh...
891952c0033111dd,Wie können Data Scientists realistisches Erwar...,"{""input"": ""Wie können Data Scientists realisti...",KI Power-Play: Realistisches Erwartungsmanagem...
1ec859b0e0c6be65,Wie tragen Data-Governance und MLOps-Techniken...,"{""input"": ""Wie tragen Data-Governance und MLOp...",Die Engineering-Perspektive auf den EU AI Act\...
5cbfb97080b2bf4b,Wie wird die Sentimentanalyse mit LLMs im Work...,"{""input"": ""Wie wird die Sentimentanalyse mit L...",Datenanalyse mit Machine Learning\n\nIn diesem...
b499be3f11cd7a3f,Welche 4 Risikogruppen für KI-Systeme erforder...,"{""input"": ""Welche 4 Risikogruppen für KI-Syste...",Die Engineering-Perspektive auf den EU AI Act\...
5cf6490c107d3236,Wie verbessert Christian Winkler die UX in NLP...,"{""input"": ""Wie verbessert Christian Winkler di...",Schluss mit Prototyp: GPTx und RAG in der Prax...
37f307177c5ac005,How do data availability and explainable AI im...,"{""input"": ""How do data availability and explai...",Datenanalysen im Fußball – ein Überblick\n\nSe...
07d83b31009459a5,How has Kira Engelhardt's finance background a...,"{""input"": ""How has Kira Engelhardt's finance b...",KI Power-Play: Realistisches Erwartungsmanagem...


In [34]:
# use extra project to see how ragas works under the hood
with using_project("ragas-evals"):
    # start evaluation
    evaluation_result = evaluate(
        dataset=ragas_eval_dataset,
        metrics=[faithfulness, answer_correctness, context_recall, context_precision],
        llm=model,
        embeddings=embeddings
    )
    
# get evaluation scores
eval_scores_df = pd.DataFrame(evaluation_result.scores)

# get evaluation data
eval_data_df = pd.DataFrame(evaluation_result.dataset)
eval_data_df

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

,question,ground_truth,answer,contexts
0,Wie bringt die interne Datenakademie der Merck...,Die interne Datenakademie der Merck KGaA bring...,"Boris Adryan, der Akademische Direktor der int...",[Wieviel KI müssen wir unseren Kunden erklären...
1,Wie hat die Einführung der Transformer-Archite...,Nach der Einführung der Transformer-Architektu...,Die Einführung der Transformer-Architektur dur...,[Eine Einführung in Large Language Models\n\nC...
2,Wie können Data Scientists realistisches Erwar...,Data Scientists können realistisches Erwartung...,Data Scientists können realistisches Erwartung...,[KI Power-Play: Realistisches Erwartungsmanage...
3,Wie tragen Data-Governance und MLOps-Techniken...,Data-Governance und MLOps-Techniken tragen zur...,"Die Anwendung von etablierten Data-Governance,...",[Die Engineering-Perspektive auf den EU AI Act...
4,Wie wird die Sentimentanalyse mit LLMs im Work...,Die Sentimentanalyse wird im Workshop mit LLMs...,Im Workshop wird die Sentimentanalyse mit LLMs...,[Datenanalyse mit Machine Learning\n\nIn diese...
5,Welche 4 Risikogruppen für KI-Systeme erforder...,Die vier Risikogruppen für KI-Systeme sind ver...,Der EU AI Act kategorisiert KI-Systeme in vier...,[Die Engineering-Perspektive auf den EU AI Act...
6,Wie verbessert Christian Winkler die UX in NLP...,Christian Winkler konzentriert sich auf die Op...,Christian Winkler beschäftigt sich als Profess...,[Schluss mit Prototyp: GPTx und RAG in der Pra...
7,How do data availability and explainable AI im...,The context discusses the impact of data avail...,The increasing availability of data and data e...,[Datenanalysen im Fußball – ein Überblick\n\nS...
8,How has Kira Engelhardt's finance background a...,Kira Engelhardt's finance background and leade...,The provided context does not provide a direct...,[KI Power-Play: Realistisches Erwartungsmanage...
9,Welche technischen Anforderungen gibt es für T...,"Die technischen Anforderungen für Teilnehmer, ...","Teilnehmer sollten einen Laptop mitbringen, de...",[Datenanalyse mit Machine Learning\n\nIn diese...


In [ ]:
# assign span ids to the ragas evaluation scores (needed so Phoenix knows where to attach the spans).
span_questions = (
    spans_dataframe[["input"]]
    .sort_values("input")
    .drop_duplicates(subset=["input"], keep="first")
    .reset_index()
    .rename({"input": "question"}, axis=1)
)

ragas_evals_df = ragas_evals_df.merge(span_questions, on="question").set_index("context.span_id")
eval_data_df = eval_data_df.merge(span_questions, on="question").set_index("context.span_id")
eval_scores_df.index = eval_data_df.index
eval_scores_df

In [ ]:
for eval_name in eval_scores_df.columns:
    evals_df = eval_scores_df[[eval_name]].rename(columns={eval_name: "score"})
    evals = SpanEvaluations(eval_name, evals_df)
    px.Client().log_evaluations(evals)

In [ ]:
px.close_app()